# Extraindo informações do perfil do Instagram com Instaloader

In [32]:
!pip3 install -q -U instaloader
!pip3 install -q -U google-generativeai
!pip3 install -q -U opencv-python

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import instaloader
import requests

# Criando uma instância do Instaloader
L = instaloader.Instaloader()

# Nome do perfil
profile_name = 'naflorestaglamping'
video_filename_uploaded = 'video_3.mp4'
prompt_final = 'Conforme exemplo anterior, elabore uma descrição para o vídeo'
prompt_video_1 = 'video_1.mp4'
prompt_video_2 = 'video_2.mp4'
prompt_video_3 = 'video_3.mp4'

# Carregar um perfil
profile = instaloader.Profile.from_username(L.context, profile_name)

# Guarda o nome do perfil
profile_name = profile.username

# Guarda a biografia do perfil
profile_biography = profile.biography

# Inicializa uma lista para guardar as descrições das postagens
descricoes = []

# Itera sobre as 10 últimas postagens do perfil
for post in profile.get_posts():
    if len(descricoes) < 10:
        # Adiciona a descrição de cada postagem na lista
        descricoes.append(post.caption)
    else:
        break

# Inicializa uma lista para guardar informações dos vídeos
videos_info = []

# Itera sobre as postagens do perfil que sao videos e extrai informacoes dos ultimos 3
for post in profile.get_posts():
    if post.is_video and len(videos_info) < 2:
        # Adiciona a url e descrição do vídeo na lista
        videos_info.append((post.video_url, post.caption))
    elif len(videos_info) >= 2:
        break

# Fazendo download e salvando os vídeos localmente
lista_videos_descricao = []
for i, (url, descricao) in enumerate(videos_info, 1):
    # Nome do arquivo onde o vídeo será salvo
    video_filename = f"video_{i}.mp4"
    
    # Fazendo o download do vídeo
    response = requests.get(url)
    
    # Salvando o vídeo no local especificado
    with open(video_filename, 'wb') as video_file:
        video_file.write(response.content)

    # Adicionando o nome do arquivo à lista de vídeos
    lista_videos_descricao.append([video_filename, descricao])

    print(f"Vídeo {i} salvo como {video_filename}")

lista_videos_descricao.append([video_filename_uploaded, prompt_final])


# Enviando informações do perfil para o modelo gerador de system instruction

In [40]:
#Configurações iniciais
import google.generativeai as genai

GOOGLE_API_KEY="SUA_API_KEY_AQUI"
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

system_instruction = "O objetivo é gerar uma system instruction para outro modelo que sera especializado em gerar descrições atraentes e coerentes para novos vídeos do perfil no Instagram. Mande apenas a system isntruction sem titulo ou observacoes, com no maximo 400 caracteres, nao utlizar links. "

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                system_instruction=system_instruction,
                                  safety_settings=safety_settings,)

# Preparando o prompt com as informações coletadas

# Adiciona o nome do perfil e a biografia ao prompt
prompt = f"Nome do perfil: {profile_name}\n"
prompt += f"Biografia: {profile_biography}\n\n"

# Adiciona as descrições das postagens ao prompt
prompt += "Descrições das Postagens:\n"
for i, descricao in enumerate(descricoes, 1):
    prompt += f"\n{i}. {descricao}\n"

response = model.generate_content(prompt)
aux_description_model_final = "Esteja pronto para ajustar seu estilo com exemplos de few-shot learning. Mande apenas a descricao, sem titulos ou observacoes"

plim_ai_instruction = response.text + aux_description_model_final

# Preparando modelo gerador de descricoes

In [42]:
import cv2
import os
import shutil

# Criando o modelo especializado em gerar descrições atraentes e coerentes para novos vídeos do perfil no Instagram
model_plim_ai = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=plim_ai_instruction,
                              safety_settings=safety_settings)

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "./content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")



# Upando frames dos videos para o Gemini

In [ ]:
from itertools import islice

# Função para obter itens distribuídos uniformemente da lista
def get_uniformly_distributed_items(lst, num_items):
    step = len(lst) / num_items
    return [lst[int(i * step)] for i in range(num_items)]

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

uploaded_files = []
for video in lista_videos_descricao:
    print(f"Processing video: {video[0]}")
    video_file_path = video[0]
    extract_frame_from_video(video_file_path)

    # Process each frame in the output directory
    files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
    files = sorted(files)
    files_to_upload = []
    for file in files:
      files_to_upload.append(
          File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))
    print(files_to_upload)    # Upload the files to the API
    # Only upload a 10 second slice of files to reduce upload time.
    # Change full_video to True to upload the whole video.
    full_video = False

    
    print(f'Uploading {len(files_to_upload) if full_video else 10} files. This might take a bit...')

    # Obtenha 10 itens distribuídos uniformemente
    files_to_upload = get_uniformly_distributed_items(files_to_upload, 10)

    for file in files_to_upload if full_video else files_to_upload:
      print(f'Uploading: {file.file_path}...')
      response = genai.upload_file(path=file.file_path)
      file.set_file_response(response)
      uploaded_files.append([video[0],file.timestamp, file.response])

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")
print(uploaded_files)

In [ ]:
instruction_1=[]
instruction_2=[]
instruction_3=[]

for file in uploaded_files:
  if file[0] == prompt_video_1:
    instruction_1.append(file[1])
    instruction_1.append(file[2])
  elif file[0] == prompt_video_2:
    instruction_2.append(file[1])
    instruction_2.append(file[2])
  elif file[0] == prompt_video_3:
    instruction_3.append(file[1])
    instruction_3.append(file[2])

instruction_1.append(lista_videos_descricao[0][1])
instruction_2.append(lista_videos_descricao[1][1])
    

print(instruction_1)
print(instruction_2)
print(instruction_3)

In [ ]:
chat = model_plim_ai.start_chat(history=[
  {
    "role": "user",
    "parts": instruction_1},
  {
    "role": "user",
    "parts": instruction_2
  },
  {
    "role": "user",
    "parts": instruction_3
  },
  
])

chat.send_message(prompt_final)

In [ ]:
chat.last.text